In [1]:
import os

os.environ["CMAKE_ARGS"] = "LLAMA_CUBLAS=on"
os.environ["FORCE_CMAKE"] = "1"

In [ ]:
#!pip install llama-index-embeddings-huggingface

In [ ]:
#!pip install --force-reinstall llama-cpp-python
#!pip install llama-index-llms-llama-cpp

In [4]:
#!pip list | grep torch

torch                              2.2.0


In [2]:
import torch
from llama_index.llms.llama_cpp import LlamaCPP

from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [4]:

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_url="/home/manifold/Desktop/DAP/LIRNEgpt/llama_index/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

In [ ]:
#!conda install pypdf
#!conda install torch
#!conda install transformers
#!conda install sentence-transformers
#!conda install llama-index

In [4]:
from llama_index import SimpleDirectoryReader
from llama_index import Document

documents = SimpleDirectoryReader(
    input_files = ["LA_data.pdf"]
).load_data()

In [5]:
documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [32]:
#!git clone https://github.com/run-llama/llama_index.git

In [33]:
#!cd llama_index

In [34]:
#!python3 setup.py install

In [6]:
Creating vector store
'''
import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

 ''' 

In [5]:
import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
def get_build_index(llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
  )

  return index

In [6]:
from llama_index import load_index_from_storage, StorageContext

In [7]:
# get the vector index
vector_index = get_build_index(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")

In [8]:
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [9]:
from llama_index.postprocessor import MetadataReplacementPostProcessor
from llama_index.postprocessor import SentenceTransformerRerank

In [10]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

In [11]:
response = query_engine.query("Who agreed to provide consumption data for the Lacuna project?")
print(response)

 LIRNEasia agreed to provide electricity consumption data for the Lacuna project.


In [12]:
response = query_engine.query("What is Lacuna?")
print(response)

 Lacuna is an organization or fund that aims to build machine learning datasets, specifically at the intersection of Climate Change and energy. LIRNEasia is proposing a project to contribute to this effort by combining electricity consumption data with longitudinal survey data on the drivers of electricity consumption from 4,000 households in Sri Lanka. Chanuka, an individual mentioned in the context, is involved in this project and other related projects using machine learning techniques for public policy applications, including satellite imagery data for climate change studies and urban planning, and call detail records for mapping socioeconomic status.


In [13]:
response = query_engine.query("Can you give me a brief description / work plan / outputs of this Electricity consumption project?")
print(response)

 Based on the context provided, LIRNEasia's proposed project involves collecting electricity consumption data for 20 months and conducting a longitudinal survey on 4,000 households in Asia to identify the drivers of electricity consumption. The organization's extensive experience in designing, implementing, and analyzing large-scale surveys since 2005, along with its expertise in the energy sector since 2007, makes it uniquely qualified to carry out this work. The outputs of the project will likely include comprehensive research findings and data sets that can be disseminated to decision-makers, policy makers, regulators, service providers, civil society, and the media to inform climate change and energy policies and solutions.


In [14]:
response = query_engine.query("What are the research work LIRNEasia conducted in Myanmar?")
print(response)

 LIRNEasia has conducted significant research in Myanmar, focusing on various aspects of digital development and policy. They were one of the first civil society organizations to study and help guide the liberalization process of the telecoms sector in Myanmar, contributing to shaping the regulatory environment for telecommunications in the country. Additionally, they have been involved in the study of platform-related issues in Myanmar, likely referring to the impact of digital platforms on society, governance, and the economy in the country.


In [ ]:
response = query_engine.query("")
print(response)

In [ ]:
query_engine.query("")

In [80]:
response = query_engine.query("How can I reach the LIRNEasia office in Sri Lanka?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     239.00 ms
llama_print_timings:      sample time =      67.57 ms /   120 runs   (    0.56 ms per token,  1775.88 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1493.51 ms /   120 runs   (   12.45 ms per token,    80.35 tokens per second)
llama_print_timings:       total time =    2108.69 ms /   121 tokens


 To reach the LIRNEasia office in Sri Lanka, you can use the following contact information:

Address: No. 24, Havelock Road, Colombo 05, Sri Lanka
Phone: +94 11 2691052 / +94 11 2691053
Email: info@lirneasia.org
Website: www.lirneasia.org

You can also check their website for more specific directions or transportation options if needed.
